# Importing Data

In [1]:
degree = 4

In [2]:
import csv
X = []
y = []
with open('data.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        [y_, *x] = [float(n) for n in row]
        y.append(y_)
        X.append(x)

# Generating Polynomial-to-Linear Transformation
(sklearn doesn't offer a multivariate polynomial regression, but we can trivially transform a polynomial regression into a linear one)

In [3]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=degree)
new_X = poly.fit_transform(X)

# Performing Regression

In [4]:
from sklearn import linear_model
reg = linear_model.LinearRegression(copy_X=True)
reg.fit(new_X, y)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

# Verifying Results

In [5]:
(reg.coef_, reg.intercept_)

(array([ 0.00000000e+00, -3.21648136e-03, -4.10024324e-03, -4.31494669e-02,
        -1.44876293e-02,  1.32170240e-04, -1.77566404e-03,  5.02757764e-07,
        -3.67441356e-06,  1.78293968e-04,  5.73347234e-04,  3.11519481e-06,
        -9.36190566e-05, -1.90657229e-08, -8.59550414e-07,  2.30348073e-05,
        -8.53008203e-12, -1.70290813e-09,  5.86708583e-08, -1.17770928e-06,
        -4.47230717e-06, -1.86761746e-07,  3.41031633e-06,  8.81553044e-10,
        -2.53449092e-09, -2.47098823e-07, -6.10204875e-14, -4.66703500e-11,
         2.96193390e-09, -3.27584383e-08, -2.13414093e-15,  2.40182808e-13,
        -3.28120075e-12, -7.95034886e-11,  1.74632333e-09]), 31.0889774990904)

In [6]:
(len(reg.coef_), reg.score(new_X, y))

(35, 0.9993504171122628)

In [7]:
# to_predict = [
#     [15,2000,120],
#     [15,2000,220],
# ]
import numpy as np
x_ = np.linspace(250,300,40)
to_predict = [[31, 5000, x] for x in x_]
new_to_predict = PolynomialFeatures(degree=degree).fit_transform(to_predict)
y_ = reg.predict(new_to_predict)

In [8]:
import matplotlib.pyplot as plt
plt.plot(x_,y_)

In [9]:
a = [np.arange(4)]
print(a)
len(PolynomialFeatures(degree=degree).fit_transform(a)[0])

[array([0, 1, 2, 3])]


70

In [10]:
reg.predict(PolynomialFeatures(degree=degree).fit_transform([[31,5000,250]]))

array([7.89550304])

In [11]:
reg.coef_


array([ 0.00000000e+00, -3.21648136e-03, -4.10024324e-03, -4.31494669e-02,
       -1.44876293e-02,  1.32170240e-04, -1.77566404e-03,  5.02757764e-07,
       -3.67441356e-06,  1.78293968e-04,  5.73347234e-04,  3.11519481e-06,
       -9.36190566e-05, -1.90657229e-08, -8.59550414e-07,  2.30348073e-05,
       -8.53008203e-12, -1.70290813e-09,  5.86708583e-08, -1.17770928e-06,
       -4.47230717e-06, -1.86761746e-07,  3.41031633e-06,  8.81553044e-10,
       -2.53449092e-09, -2.47098823e-07, -6.10204875e-14, -4.66703500e-11,
        2.96193390e-09, -3.27584383e-08, -2.13414093e-15,  2.40182808e-13,
       -3.28120075e-12, -7.95034886e-11,  1.74632333e-09])

# Generating Code
(the first cell produces the C expression that evaluates to the result of the regression given the coefficients and the `x`, `v`, and `t` values; the second cell produces the C expression representing the coefficients)

In [12]:
the_list = zip(
    ['c['+str(i)+']' for i in range(len(reg.coef_))], 
    poly.get_feature_names(['t', 'x', 'v'])
)

' + '.join([
    v[0] + ' * ' + (
        ' * '.join(v[1].replace('^', '_').split(' '))
    ) for v in the_list   
])

'c[0] * 1 + c[1] * t + c[2] * x + c[3] * v + c[4] * t_2 + c[5] * t * x + c[6] * t * v + c[7] * x_2 + c[8] * x * v + c[9] * v_2 + c[10] * t_3 + c[11] * t_2 * x + c[12] * t_2 * v + c[13] * t * x_2 + c[14] * t * x * v + c[15] * t * v_2 + c[16] * x_3 + c[17] * x_2 * v + c[18] * x * v_2 + c[19] * v_3 + c[20] * t_4 + c[21] * t_3 * x + c[22] * t_3 * v + c[23] * t_2 * x_2 + c[24] * t_2 * x * v + c[25] * t_2 * v_2 + c[26] * t * x_3 + c[27] * t * x_2 * v + c[28] * t * x * v_2 + c[29] * t * v_3 + c[30] * x_4 + c[31] * x_3 * v + c[32] * x_2 * v_2 + c[33] * x * v_3 + c[34] * v_4'

In [13]:
coefs = reg.coef_
coefs[0] = reg.intercept_
'float c[] = {' + (', '.join([(str(v).replace('e', 'E')) for v in reg.coef_])) + '};'

'float c[] = {31.0889774990904, -0.003216481364462731, -0.004100243239694015, -0.04314946689850364, -0.014487629338360983, 0.00013217024013931553, -0.001775664037136168, 5.027577642934903E-07, -3.6744135645886206E-06, 0.00017829396790668568, 0.0005733472337225401, 3.1151948099235467E-06, -9.361905662145324E-05, -1.9065722948631668E-08, -8.595504140745983E-07, 2.3034807328705295E-05, -8.530082029710936E-12, -1.7029081294710465E-09, 5.8670858286297504E-08, -1.1777092799530631E-06, -4.472307173112913E-06, -1.8676174554666065E-07, 3.4103163258855264E-06, 8.815530438031664E-10, -2.5344909211140704E-09, -2.470988233043669E-07, -6.102048752446398E-14, -4.66703500405816E-11, 2.9619338971249244E-09, -3.275843828840046E-08, -2.1341409279473684E-15, 2.4018280822876314E-13, -3.2812007493696147E-12, -7.950348864514952E-11, 1.7463233288448203E-09};'